In [11]:
import sys; sys.path.append('../../')
import numpy as np
from DataPreparation.DataPreparation import read_data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from mlpath import mlquest as mlq
mlq.start_quest("LogisticRegression", table_dest="../../", log_defs=False)

### Read the data

In [12]:
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical')

### Initiate model

In [13]:
clf = mlq.l(LogisticRegression)(multi_class='multinomial')

### Train and Evaluate

In [14]:
y_pred = mlq.l(cross_val_predict)(clf, x_data_d, y_data_d, cv=4)
accuracy = np.mean(y_pred == y_data_d)
mlq.log_metrics(accuracy=accuracy.item())
print(classification_report(y_data_d, y_pred))
mlq.end_quest()

              precision    recall  f1-score   support

           0       0.86      0.99      0.92       152
           1       0.97      0.58      0.73       160
           2       0.87      0.99      0.92       324
           3       1.00      0.99      0.99       544

    accuracy                           0.93      1180
   macro avg       0.92      0.89      0.89      1180
weighted avg       0.94      0.93      0.93      1180



### Tracking

In [15]:
mlq.show_logs("LogisticRegression", table_dest="../../", last_k=6)